In [2]:
import os
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import cv2
import random
import time
from PIL import Image
import pandas as pd

In [3]:
# Library Constants
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkPoints = mp.solutions.hands.HandLandmark
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
DrawingUtil = mp.solutions.drawing_utils

In [4]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

base_options = BaseOptions(model_asset_path='data/hand_landmarker.task')
options = HandLandmarkerOptions(base_options=base_options,
                                        num_hands=2)

detector = HandLandmarker.create_from_options(options)
def convert_detection_result(detection_result, image):
        """
        Draws all the landmarks on the hand
        Args:
            image (Image): Image to draw on
            detection_result (HandLandmarkerResult): HandLandmarker detection results
        """
        # Get a list of the landmarks
        hand_landmarks_list = detection_result.hand_landmarks
        
        for idx in range(len(hand_landmarks_list)):
            #NOTE this only works for one hand
            listofpointstoreturn = []
            hand_landmarks = hand_landmarks_list[idx]

            # Save the landmarks into a NormalizedLandmarkList
            hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
            hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
            ])
            # results = mp_hands.Hands.process(image = image)

            if hand_landmarks:
                for landmrk in hand_landmarks:
                    # for ids, landmrk in enumerate(hand_landmarks.landmark):
                        # print(ids, landmrk)
                    listofpointstoreturn.append(landmrk.x)
                    listofpointstoreturn.append(landmrk.y)
                    listofpointstoreturn.append(landmrk.z)
                return listofpointstoreturn
        
        return "Nothing Here"

I0000 00:00:1715138571.179610       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
def returndatapoints(image):
    to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    results = detector.detect(to_detect)            
    # Draw the hand landmarks
    points = convert_detection_result(results, image)
    if points == "Nothing Here":
        border_size = 150
        frame = cv2.copyMakeBorder(image, top=border_size, 
                                bottom=border_size, left=border_size, 
                                right=border_size,
                                borderType=cv2.BORDER_CONSTANT, 
                                value=[0, 0, 0]
        )
        to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        results = detector.detect(to_detect) 
        points = convert_detection_result(results, frame)
    return points


In [6]:
def draw_landmarks_on_hand(image, detection_result):
    """
    Draws all the landmarks on the hand
    Args:
        image (Image): Image to draw on
        detection_result (HandLandmarkerResult): HandLandmarker detection results
    """
    # Get a list of the landmarks
    hand_landmarks_list = detection_result.hand_landmarks

    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]

        # Save the landmarks into a NormalizedLandmarkList
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        # Draw the landmarks on the hand
        DrawingUtil.draw_landmarks(image,
                                    hand_landmarks_proto,
                                    solutions.hands.HAND_CONNECTIONS,
                                    solutions.drawing_styles.get_default_hand_landmarks_style(),
                                    solutions.drawing_styles.get_default_hand_connections_style())


In [7]:
"""
Main game loop. Runs until the 
user presses END_LETTER.
"""    
END_LETTER = "q"
# Create the hand detector
base_options = BaseOptions(model_asset_path='data/hand_landmarker.task')
options = HandLandmarkerOptions(base_options=base_options,
                                        num_hands=2)
detector = HandLandmarker.create_from_options(options)

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
total_data_collector = []
CURRENT_LETTER = "f"
NUM_DATA = 30


# TODO: Load video
video = cv2.VideoCapture(0)
START_TIME = time.time()
# TODO: Modify loop condition  
while video.isOpened():
    # Get the current frame
    frame = video.read()[1]

    # Convert it to an RGB image
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)
    # Convert the image to a readable format and find the hands
    to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    results = detector.detect(to_detect)            
    

    # Draw the hand landmarks
    draw_landmarks_on_hand(image, results)
    datapoints = returndatapoints(image)
    if datapoints != "Nothing Here":
        most_recent_actual_data = datapoints

    # Change the color of the frame back
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imshow("Hand Tracking", image)

    #stops data collection at NUM_DATA
    if len(total_data_collector) >= NUM_DATA:
        break
    # Break the loop if the user presses 'q'
    if cv2.waitKey(50) & 0xFF == ord(END_LETTER):
        break
    if cv2.waitKey(50) & 0xFF == ord(CURRENT_LETTER):
        #print(most_recent_actual_data)
        print(len(total_data_collector))
        if datapoints != "Nothing Here":
            total_data_collector.append(most_recent_actual_data)
            
            

video.release()
cv2.destroyAllWindows()
#cv2.imshow("Hand Tracking", image)



I0000 00:00:1715138571.263361       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


KeyboardInterrupt: 

In [100]:
len(total_data_collector)

30

In [101]:
#total_data_collector[2]

In [102]:
acquired_df = []
copy_of_total_data = total_data_collector
current_letter = CURRENT_LETTER
for i in range(len(copy_of_total_data)):
    copy_of_total_data[i].append(current_letter)
    acquired_df.append(copy_of_total_data[i])



In [103]:
dfcolnames = []
for i in range(1,22):
    for j in ["x","y","z"]:
        dfcolnames.append(j+str(i))
dfcolnames.append("label")

In [104]:
copy_df = acquired_df

In [105]:
#copy_df

In [106]:
# out_list = []
# for i in range(len(copy_df)):
#     out_list.append(copy_df[i][0])

In [107]:
end_df = pd.DataFrame(copy_df, columns = dfcolnames)

In [108]:
#end_df

In [112]:
df  = pd.read_csv('data/acquired_data/' + CURRENT_LETTER + '.csv')

In [110]:
#df

In [111]:
end_df = pd.concat([df, end_df], ignore_index = True)
# end_df.reset_index()

In [97]:
#end_df

In [114]:
end_df.to_csv('data/acquired_data/' + CURRENT_LETTER + '.csv', index=False) 
              

In [126]:
# copy_df[0]

In [53]:
#end_df

In [20]:
"""
Main game loop. Runs until the 
user presses END_LETTER.
"""    
END_LETTER = "q"
# Create the hand detector
base_options = BaseOptions(model_asset_path='data/hand_landmarker.task')
options = HandLandmarkerOptions(base_options=base_options,
                                        num_hands=2)
detector = HandLandmarker.create_from_options(options)

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
total_data_collector = []
CURRENT_LETTER = "y"
NUM_DATA = 30


video = cv2.VideoCapture(0)
START_TIME = time.time()
while video.isOpened():
    # Get the current frame
    frame = video.read()[1]

    # Convert it to an RGB image
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)
    # Convert the image to a readable format and find the hands
    to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    results = detector.detect(to_detect)            
    

    # Draw the hand landmarks
    draw_landmarks_on_hand(image, results)
    datapoints = returndatapoints(image)
    if datapoints != "Nothing Here":
        most_recent_actual_data = datapoints

    # Change the color of the frame back
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imshow("Hand Tracking", image)

    #stops data collection at NUM_DATA
    if len(total_data_collector) >= NUM_DATA:
        break
    # Break the loop if the user presses 'q'
    if cv2.waitKey(50) & 0xFF == ord(END_LETTER):
        break
    if cv2.waitKey(50) & 0xFF == ord(CURRENT_LETTER):
        #print(most_recent_actual_data)
        print(len(total_data_collector))
        if datapoints != "Nothing Here":
            total_data_collector.append(most_recent_actual_data)
            
            

video.release()
cv2.destroyAllWindows()
#cv2.imshow("Hand Tracking", image)
acquired_df = []
copy_of_total_data = total_data_collector
current_letter = CURRENT_LETTER
for i in range(len(copy_of_total_data)):
    copy_of_total_data[i].append(current_letter)
    acquired_df.append(copy_of_total_data[i])
dfcolnames = []
for i in range(1,22):
    for j in ["x","y","z"]:
        dfcolnames.append(j+str(i))
dfcolnames.append("label")
copy_df = acquired_df
end_df = pd.DataFrame(copy_df, columns = dfcolnames)
df  = pd.read_csv('data/acquired_data/' + CURRENT_LETTER + '.csv')
end_df = pd.concat([df, end_df], ignore_index = True)
end_df.to_csv('data/acquired_data/' + CURRENT_LETTER + '.csv', index=False) 


I0000 00:00:1715139833.667164       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
